In [1]:
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10, STL10
from torch.utils.data import DataLoader

In [2]:
torch.cuda.is_available()

True

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((227)),
    transforms.CenterCrop((227, 227)),
])

cifar10_classes = CIFAR10(root='./data', train=True, download=True).classes

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [11]:
from torchvision.utils import make_grid
import matplotlib.pyplot as plt

num = 5

# def show_batch(batch_images):
#     fig,ax = plt.subplots(figsize=(16,12))
#     ax.set_xticks([])
#     ax.set_yticks([])
#     ax.imshow(make_grid(batch_images,nrow=16).permute(1,2,0))

# batch = next(iter(train_loader))
# show_batch(batch)
# print(batch[1])

# image = batch[0][0]
image.shape

torch.Size([3, 227, 227])

In [12]:
from torch import nn

class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        
        self.l1 = nn.Sequential(
            nn.Conv2d(3, 96, 11, stride=4, padding=0), # in, out, kernel_size
            nn.BatchNorm2d(num_features=96),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.l2 = nn.Sequential(
            nn.Conv2d(96, 256, 5, stride=1, padding=2),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.l3 = nn.Sequential(
            nn.Conv2d(256, 384, 3, stride=1, padding=1),
            nn.ReLU(),
        )
        self.l4 = nn.Sequential(
            nn.Conv2d(384, 384, 3, stride=1, padding=1),
            nn.ReLU(),
        )
        self.l5 = nn.Sequential(
            nn.Conv2d(384, 256, 3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )

        self.fc1 = nn.Sequential(
            nn.Linear(256*6*6, 4096), # in, out
            nn.ReLU(),
        )
        self.fc2 = nn.Sequential(
            nn.Linear(4096, 4096),
            nn.ReLU(),
        )
        self.fc3 = nn.Linear(4096, num_classes)

    
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x = self.l5(x)

        x = x.view(-1, 256*6*6)

        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)

        return x

In [13]:
alexnet = AlexNet()

In [14]:
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss()
optimiser = optim.SGD(alexnet.parameters(), momentum=0.9, lr=0.005, weight_decay=0.005)

In [16]:
import torch

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Move the model to CUDA
alexnet.to(device)

Device: cuda


AlexNet(
  (l1): Sequential(
    (0): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
    (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (l2): Sequential(
    (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (l3): Sequential(
    (0): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (l4): Sequential(
    (0): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
  )
  (l5): Sequential(
    (0): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=Fals

In [17]:
# Inside the training loop
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to GPU

        optimiser.zero_grad()

        outputs = alexnet(inputs)
        labels = labels.squeeze()

        loss = criterion(outputs, labels)
        loss.backward()
        optimiser.step()

        running_loss += loss.item()
        if i % 20 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 20))
            running_loss = 0.0

print('Finished Training')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!